<a href="https://colab.research.google.com/github/isaacmaruyama/nightlights/blob/main/blackmarble_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Overview
This is a Python script I wrote that uses nightlights data taken from NASA's Black Marble program to examine trends in luminosity for specific Chinese administrative states.



In [ ]:
# Import library dependencies
import osgeo.gdal as gdal, numpy as np, os, ee, geemap

# Earth Engine authentification
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=4iSPSCw6vNJV5mX_DeplgFdkLnf41gngW2ZecOAeEuA&tc=q2lTiEhqiNW9ehFc9EZ3kmR98L10_BYyOtLVoaoeJd0&cc=EV4jyMURlIUKgt8Ap2Mxo-qGBvUAsYJzMvhs0qo-mcQ

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXm6eUxxfgivu9BKcqovrsB4onAV5ZcGdolW1buQm8v-vm2RUV7rlgw

Successfully saved authorization token.


In [ ]:
# Set time variables for luminosity comparison
month = '09' # 2-digit format
comparison_years = [2022, 2023] # 2 years

# Create empty lists for comparing radiance levels
old_list = []
new_list = []

# Load the China states feature collection
china_states = ee.FeatureCollection('FAO/GAUL/2015/level1') \
    .filter(ee.Filter.eq('ADM0_NAME', 'China'))

# Define style parameters for visualization
style_params = {'color': 'white', 'width': 1, 'fillColor': '00000000'}

# Define counter variable
counter = 0

# Generate radiance data for each comparison period
for year in comparison_years:
  # Define the time range for filtering the image collection
  start_date = f'{year}-{month}-01'
  end_date = f'{year}-{month}-20'

  # Load the Nightlights image collection
  nightlights = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG') \
      .filter(ee.Filter.date(start_date, end_date)) \
      .select('avg_rad')

  # Define visualization parameters for nightlights
  nightlight_vis_params = {'min': 0.0, 'max': 60.0}

  # Create a mosaic of nightlights and clip it to China states
  mosaic_nightlights = nightlights.mosaic()
  clipped_nightlights = mosaic_nightlights.clip(china_states)

  # Reduce regions to compute mean radiance within each China state
  average_radiance = clipped_nightlights.reduceRegions(
      collection=china_states,
      reducer=ee.Reducer.mean(),
      scale=1000  # Adjust the scale based on your data resolution
  )

  # Map the extraction function over the feature collection
  extracted_data = average_radiance.map(lambda feature: feature.select(['ADM1_NAME', 'mean'])
      .set({
          'name': feature.get('ADM1_NAME'),
          'mean': feature.get('mean')
  }))

  # Get the features to print
  features_to_print = extracted_data.getInfo()['features']

  # Loop through each feature and add its properties to the list
  for feature in features_to_print:
      properties = feature['properties']
      name_value = properties['name']
      mean_value = properties['mean']

      # Add properties to the list as a tuple
      if counter == 0:
        old_list.append((name_value, mean_value))
      elif counter == 1:
        new_list.append((name_value, mean_value))

  counter += 1

# Make comparison list
difference_list = [(name, (value2 - value1)/value1) for (name, value1), (_, value2) in zip(old_list, new_list)]

In [ ]:
print(difference_list)

[('Anhui Sheng', -0.09349946487636573), ('Beijing Shi', -0.022815718812826704), ('Chongqing Shi', 0.03904718942641669), ('Fujian Sheng', 0.09806465724124307), ('Gansu Sheng', 0.10941124429069228), ('Guangdong Sheng', 0.04782411818116257), ('Guangxi Zhuangzu Zizhiqu', 0.09661702674312798), ('Guizhou Sheng', 0.17494570270158372), ('Hainan Sheng', 0.12563631374170423), ('Hebei Sheng', 0.03286210539258551), ('Henan Sheng', -0.04586821262075667), ('Hubei Sheng', -0.01882396546120645), ('Hunan Sheng', -0.02370029272764351), ('Jiangsu Sheng', -0.02927370452385976), ('Jiangxi Sheng', 0.11729226171020815), ('Liaoning Sheng', -0.014026846953567077), ('Nei Mongol Zizhiqu', 0.05092549979450901), ('Ningxia Huizu Zizhiqu', 0.2561426162239362), ('Qinghai Sheng', 0.15896785321813064), ('Shaanxi Sheng', 0.0268346061308152), ('Shandong Sheng', 0.020014878679431997), ('Shanghai Shi', -0.019721221281452126), ('Shanxi Sheng', 0.016449584511891477), ('Sichuan Sheng', 0.6643025693356956), ('Tianjin Shi', -0.

In [ ]:
# Create color mapping function
def create_custom_colormap(integer_list):
    max_abs_value = max(abs(max(integer_list)), abs(min(integer_list)))

    # Define a custom colormap going from blue to red
    colors = [(0, 0, 1), (1, 0, 0)]  # Blue to Red

    # Map normalized values to colors using the custom colormap
    colormap_values = [(int(c[0] * 255), int(c[1] * 255), int(c[2] * 255)) for c in np.linspace(colors[0], colors[1], len(integer_list))]

    return colormap_values

integer_list = [entry[1] for entry in difference_list]

# Create color translation function
def rgb_to_hex(rgb):
    return "#{:02x}{:02x}{:02x}".format(rgb[0], rgb[1], rgb[2])

# Create a custom colormap using the blue-to-white-to-red scheme
colormap_values = create_custom_colormap(integer_list)

hex_colors = [rgb_to_hex(rgb) for rgb in colormap_values]

# Combine the original data with the new RGB color values
result_data = [(name, value, color) for (name, value), color in zip(difference_list, hex_colors)]

print(result_data)

[('Anhui Sheng', -0.09349946487636573, '#0000ff'), ('Beijing Shi', -0.022815718812826704, '#0800f6'), ('Chongqing Shi', 0.03904718942641669, '#1100ee'), ('Fujian Sheng', 0.09806465724124307, '#1900e5'), ('Gansu Sheng', 0.10941124429069228, '#2200dd'), ('Guangdong Sheng', 0.04782411818116257, '#2a00d4'), ('Guangxi Zhuangzu Zizhiqu', 0.09661702674312798, '#3300cc'), ('Guizhou Sheng', 0.17494570270158372, '#3b00c3'), ('Hainan Sheng', 0.12563631374170423, '#4400bb'), ('Hebei Sheng', 0.03286210539258551, '#4c00b2'), ('Henan Sheng', -0.04586821262075667, '#5500aa'), ('Hubei Sheng', -0.01882396546120645, '#5d00a1'), ('Hunan Sheng', -0.02370029272764351, '#660099'), ('Jiangsu Sheng', -0.02927370452385976, '#6e0090'), ('Jiangxi Sheng', 0.11729226171020815, '#770088'), ('Liaoning Sheng', -0.014026846953567077, '#7f007f'), ('Nei Mongol Zizhiqu', 0.05092549979450901, '#880077'), ('Ningxia Huizu Zizhiqu', 0.2561426162239362, '#90006e'), ('Qinghai Sheng', 0.15896785321813064, '#990066'), ('Shaanxi S

In [ ]:
# Create color mapping function for positive values - TEST
def create_positive_colormap(integer_list):
    # Define a custom colormap going from white to red
    colors = [(1, 1, 1), (1, 0, 0)]  # White to Red

    # Map normalized values to colors using the custom colormap
    colormap_values = [(int(c[0] * 255), int(c[1] * 255), int(c[2] * 255)) for c in np.linspace(colors[0], colors[1], len(integer_list))]

    return colormap_values

# Rest of your code remains unchanged
positive_integer_list = [value for value in integer_list if value >= 0]

# Create color translation function
def rgb_to_hex(rgb):
    return "#{:02x}{:02x}{:02x}".format(rgb[0], rgb[1], rgb[2])

# Create a custom colormap for positive values using the white-to-red scheme
positive_colormap_values = create_positive_colormap(positive_integer_list)

# Extend the colormap to include negative values (set to blue)
colormap_values = [(0, 0, 255)] * (len(integer_list) - len(positive_integer_list)) + positive_colormap_values

hex_colors = [rgb_to_hex(rgb) for rgb in colormap_values]

# Combine the original data with the new RGB color values
result_data = [(name, value, color) for (name, value), color in zip(difference_list, hex_colors)]

print(result_data)

In [ ]:
# Create a Map instance using geemap
Map = geemap.Map(center=[35, 105], zoom=4)  # Adjust the center and zoom level as needed

Map.addLayer(clipped_nightlights, nightlight_vis_params, 'Nightlights')

# Create separate layers for each state, colored based on radiance following the previously generated colormap
for item in result_data:
    name, radiance, color = item
    selected_feature = china_states.filter(ee.Filter.eq('ADM1_NAME', name))
    Map.addLayer(selected_feature, {'color': color}, name)

Map.addLayerControl()

# View map
Map

Map(center=[35, 105], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chi…

In [ ]:
# Export the extracted data to a CSV file
"""
export_params = {

    'collection': extracted_data,
    'description': 'exported_data',
    'fileFormat': 'CSV'
}
"""

"\nexport_params = {\n\n    'collection': extracted_data,\n    'description': 'exported_data',\n    'fileFormat': 'CSV'\n}\n"

In [ ]:
Map = geemap.Map()

legend_keys = ['One', 'Two', 'Three', 'Four', 'ect']
# colorS can be defined using either hex code or RGB (0-255, 0-255, 0-255)
legend_colors = ['#8DD3C7', '#FFFFB3', '#BEBADA', '#FB8072', '#80B1D3']
# legend_colors = [(255, 0, 0), (127, 255, 0), (127, 18, 25), (36, 70, 180), (96, 68 123)]

Map.add_legend(
    keys=legend_keys, colors=legend_colors, position='bottomleft'
)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…